In [1]:
import mne
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report, write_raw_bids
import pandas as pd

In [2]:
bids_root = r"C:\Users\Jonathan\Charité - Universitätsmedizin Berlin\Interventional Cognitive Neuromodulation - Data\BIDS_Berlin_ECOG_LFP\rawdata"

subject = '010'
datatype = 'ieeg'
session = 'EcogLfpMedOn02'
task = 'SelfpacedHandTapL'
acquisition = 'StimOff'
run = '1'
suffix = 'ieeg'

bids_path = BIDSPath(subject=subject, session=session, task=task, acquisition=acquisition, run=run,
                     suffix=suffix, datatype=datatype, root=bids_root)

print(bids_path)
raw = read_raw_bids(bids_path=bids_path, verbose=False)
raw.plot()

C:/Users/Jonathan/Charité - Universitätsmedizin Berlin/Interventional Cognitive Neuromodulation - Data/BIDS_Berlin_ECOG_LFP/rawdata/sub-010/ses-EcogLfpMedOn02/ieeg/sub-010_ses-EcogLfpMedOn02_task-SelfpacedHandTapL_acq-StimOff_run-1_ieeg.vhdr


C:\Users\Jonathan\AppData\Local\Temp/ipykernel_25872/3379741468.py:15: RuntimeWarning: Did not find any events.tsv associated with sub-010_ses-EcogLfpMedOn02_task-SelfpacedHandTapL_acq-StimOff_run-1.

The search_str was "C:\Users\Jonathan\Charité - Universitätsmedizin Berlin\Interventional Cognitive Neuromodulation - Data\BIDS_Berlin_ECOG_LFP\rawdata\sub-010\**\ieeg\sub-010_ses-EcogLfpMedOn02*events.tsv"
  raw = read_raw_bids(bids_path=bids_path, verbose=False)
C:\Users\Jonathan\AppData\Local\Temp/ipykernel_25872/3379741468.py:15: RuntimeWarning: Expected to find a single channels file associated with sub-010_ses-EcogLfpMedOn02_task-SelfpacedHandTapL_acq-StimOff_run-1, but found 2: "['C:\\Users\\Jonathan\\Charité - Universitätsmedizin Berlin\\Interventional Cognitive Neuromodulation - Data\\BIDS_Berlin_ECOG_LFP\\rawdata\\sub-010\\ses-EcogLfpMedOffOnDys01\\ieeg\\sub-010_ses-EcogLfpMedOn02_task-SelfpacedHandTapL_acq-StimOff_run-1_channels.tsv', 'C:\\Users\\Jonathan\\Charité - Universität

Using qt as 2D backend.
Free RAM space could not be determined because"psutil" is not installed. Setting precompute to False.


In [ ]:
# overview cell
# total recording time in seconds: 6600
# 1 Rest 10:54-10:58

0-300
# 2 Selfp li ???
 300-1180
# 3 Rest 3min 11:12 - ???
 1180-1340
# 4 Selfp Li
# 5 Selp re
 1340-2015
# 6 Rest 11:26 - 11:30
 2015-2220
# 7 selfp Li 11:30 - 11:37
 2220-2800
# 8 Free 11:37 - 11:48
 2800-3030
# 9 Rest 11:48-11:55
 3330-3700
# 10 Selfp Li 11:55 - 12:01
 3700-4110
# 11 Free 12:01-12:15
 4110-5000
# 12 Rest 12:15 - 12:20
 5000-5360
# 13 Selfp Li 12:20 - 12:27
# 14 Selfp Re 12:27 - 12:30
 5360-5900
# 15 Free 12:30 - 12:37
 5900-6300
# 16 Rest 12:38 - 12:43
 6300-6600



In [ ]:

# create table  for annotations
annot_table = []

#annot_table.field_names = ["task", "acquisition", "onset", "end"]


# 1 Rest 10:54-10:58
annot_table.append(['Rest','StimOffPreDopa',0,300])
# 2 Selfp li ???
annot_table.append(['SelfpacedHandTapL','StimOffPreDopa',300,1180])
# 3 Rest 3min 11:12 - ???
annot_table.append(['Rest','StimOffDopa00',1180,1340])

# 4 Selfp Li
# 5 Selp re
annot_table.append(['SelfpacedHandTapB','StimOffDopa05',1340,2015])
# 6 Rest 11:26 - 11:30
annot_table.append(['Rest','StimOffDopa15',2015,2220])

# 7 selfp Li 11:30 - 11:37
annot_table.append(['SelfpacedHandTapL','StimOffDopa20',2220,2800])

# 8 Free 11:37 - 11:48

annot_table.append(['Free','StimOffDopa25',2800,3030])

# 9 Rest 11:48-11:55

annot_table.append(['Rest','StimOffDopa35',3330,3700])

# 10 Selfp Li 11:55 - 12:01
annot_table.append(['SelfpacedHandTapL','StimOffDopa45',3700,4110])

# 11 Free 12:01-12:15
annot_table.append(['Free','StimOffDopa50',4110,5000])

# 12 Rest 12:15 - 12:20
annot_table.append(['Rest','StimOffDopa65',5000,5360])

# 13 Selfp Li 12:20 - 12:27
# 14 Selfp Re 12:27 - 12:30
annot_table.append(['SelfpacedHandTapB','StimOffDopa70',5360,5900])

# 15 Free 12:30 - 12:37
annot_table.append(['Free','StimOffDopa80',5900,6300])

# 16 Rest 12:38 - 12:43
annot_table.append(['Rest','StimOffDopa85',6300,6600])



annot_table = pd.DataFrame(annot_table, columns=["task", "acquisition", "onset", "end"])
annot_table['duration'] = annot_table['end'] - annot_table['onset']

In [ ]:

annot = mne.Annotations(onset=annot_table['onset'], duration=annot_table['duration'],
            description=annot_table['task']+annot_table['acquisition'])

raw2 = raw.copy().set_annotations(annot)
raw2.plot()

raws = raw2.crop_by_annotations(annotations=annot)


In [ ]:
for ra in raws:
    print(ra.info)
    bids_path = BIDSPath(subject=subject,
                        session="EcogLfpMedOfMedOnDys01",
                        task=task,
                        acquisition=acquisition,
                        run=run,
                        suffix=suffix,
                        datatype=datatype,
                        root=r"C:\Users\Jonathan\Documents\DATA\Dykinesia")
    write_raw_bids(ra, bids_path=bids_path)

In [ ]:
ra.info